# Battle of Neighbourhoods - Week 2

# 1. Introduction

## 1.1 Background

## 1.2 Problem

## 1.3 Interest

# 2. Data acquisition and cleaning

## 2.1 Data sources

1. We're scrapping the site <a href="https://www.point2homes.com/CA/Real-Estate-Listings/ON/Toronto.html?location=Toronto%2C+ON&search_mode=location&page=3&SelectedView=listings&LocationGeoId=783094&location_changed=&ajax=1">point2homes.com on Toronto</a> to predict house sales on Toronto Neighbourhoods.
2. Using  <a href='http://data.torontopolice.on.ca/datasets/mci-2014-to-2018'> The MCI  dataset includes all Major Crime Indicators (MCI) 2014 to 2018 occurrences by reported date and related offences </a>  to predict crime on Toronto Neighbourhoods.
3. We will be completely working on <a href='https://developer.foursquare.com/'>Foursquare data</a> to explore and try to locate our new house where more venues like church, restaurant, bar, hotel museums, memorials etc.. that are present nearby.

### 2.1.1 Importing Libraries and Data Collections

In [2]:
import warnings
warnings.filterwarnings('ignore') 
import requests # HTTP library
import pandas as pd # for data analysis
import numpy as np  # data in a vectorized manner manipulation
# Matplotlib and associated plotting modules for visualization
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import statsmodels.api as sm # implement statistic models
import time # use time
from geopy.geocoders import Nominatim  # for geocoders referencing
import geopandas as gpd # for spatial dataset
import seaborn as sns # for plotting and visulalization
from scipy import stats # statistic computation
from bs4 import BeautifulSoup # scrapping web site
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re # regualr expression
import requests

In [ ]:
# using user to querying the site web
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36 Edg/80.0.100.0'
headers={'User-Agent':user_agent} 

 We're scrapping the site <a href="https://www.point2homes.com/CA/Real-Estate-Listings/ON/Toronto.html?location=Toronto%2C+ON&search_mode=location&page=3&SelectedView=listings&LocationGeoId=783094&location_changed=&ajax=1">point2homes.com on Toronto</a> to predict house sales on Toronto Neighbourhoods.

In [ ]:
## scrapping www.point2homes.com to retreive info
long_list = []
lat_list = []
address_list = []
price_list= []
data_url_list = []
bed_list = []
bath_list = []
bath_list = []
sqft_list = []
type_list = []
# iterates in 148 pages
for index in range(1,148):
    url = 'https://www.point2homes.com/CA/Real-Estate-Listings/ON/Toronto.html?location=Toronto%2C+ON&search_mode=location&page={}&SelectedView=listings&LocationGeoId=783094&location_changed=&ajax=1'.format(index)
    page = requests.get(url,headers=headers)
    soup = BeautifulSoup(page.content,'html.parser')
    time.sleep(3)
    articles = soup.find_all('article')
           
    for article in articles:
        bed = article.find(class_ = 'ic-beds')
        if bed != None:
            bed_list.append(bed.get_text().replace('\n','').strip())
        else:
            bed_list.append('--')
        bath = article.find(class_ = 'ic-baths')
        if bath != None:
            bath_list.append(bath.get_text().replace('\n','').strip())
        else:
            bath_list.append('--')

        sqft = article.find(class_ = 'ic-sqft')
        if sqft != None:
            sqft_list.append(sqft.get_text().replace('\n','').strip())
        else:
            sqft_list.append('--')

        typ = article.find(class_ = 'ic-proptype')
        if typ != None:
            type_list.append(typ.get_text().replace('\n','').strip())
        else:
            type_list.append('--')
            
        price = article.find(class_ ='price')
        
        if price != None:
            price_list.append(price['data-price'])
        else:
            price_list.append('--')
        
        address = article.find(class_ ='item-address')
        if address != None:
            data_url_list.append(address['data-url'])
        else:
            price_list.append('--')
        
        inputs = article.find_all('input')
        if len(inputs) == 3:
            address_list.append(inputs[0]['value'])
            long_list.append(inputs[2]['value'])
            lat_list.append(inputs[1]['value'])
        elif len(inputs) == 2  :
            if 'ShortAddress' in inputs[0]['id']:
                address_list.append(inputs[0]['value'])
                long_list.append(0)
                lat_list.append(0)
            elif 'ShortAddress' in inputs[1]['id']:
                address_list.append(inputs[1]['value'])
                long_list.append(0)
                lat_list.append(0)
            else:
                address_list.append('--')
                long_list.append(0)
                lat_list.append(0)
        elif len(inputs) == 1  :
            if 'ShortAddress' in inputs[0]['id']:
                address_list.append(inputs[0]['value'])
                long_list.append(0)
                lat_list.append(0)
            else :
                address_list.append('--')
                long_list.append(0)
                lat_list.append(0)
        elif len(inputs) == 0:
            address_list.append('--')
            long_list.append(0)
            lat_list.append(0)

# use dictionary before use dataframe
data_dict = dict()
data_dict['address'] = address_list
data_dict['long'] = long_list
data_dict['lat'] = lat_list
data_dict['data_url'] = data_url_list
data_dict['price_$CAN'] = price_list
data_dict['beds'] = bed_list
data_dict['baths'] =bath_list
data_dict['sqft'] = sqft_list
data_dict['type'] = type_list
# convert dictionary to  dataframe
house_data = pd.DataFrame(data_dict)
# save dataframe as csv file
house_data.to_csv('./dataset/house_data.csv')

In [ ]:
# use the dataframe
house_data = pd.read_csv('./dataset/house_data.csv')
print(house_data.shape)
house_data.head(10)

Use <a href='http://data.torontopolice.on.ca/datasets/mci-2014-to-2018'> The MCI  dataset includes all Major Crime Indicators (MCI) 2014 to 2018 occurrences by reported date and related offences </a>  to predict crime on Toronto Neighbourhoods.

In [ ]:
mci_2014_2018 = pd.read_csv('./dataset/mci_2014_to_2018.csv')
print(mci_2014_2018.shape)
mci_2014_2018.head(10)

3. We will be completely working on <a href='https://developer.foursquare.com/'>Foursquare data</a> to explore and try to locate our new house where more venues like church, restaurant, bar, hotel museums, memorials etc.. that are present nearby.

In [ ]:
CLIENT_ID = 'XXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXX' # your Foursquare Secret
VERSION = '20191028'
LIMIT = 150

latitude = house_data.head(1).lat.values[0]  
longitude =house_data.head(1).long.values [0]
toronto='Toronto location : {},{}'.format(latitude,longitude)
print(toronto)

In [ ]:
#Quering for hotel & restaurant

search_query = 'hotel'
search_query_res = 'restaurant'

radius = 1000
url_hotel = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url_restaurant = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_res, radius, LIMIT)
#url

In [ ]:
results_hotel = requests.get(url_hotel).json()
results_restaurant = requests.get(url_restaurant).json()


In [ ]:
results_hotel

In [ ]:
results_restaurant

## 2.2 Data cleaning

In [31]:
# use the dataframe
house_data = pd.read_csv('./dataset/house_data.csv')
print(house_data.shape)
house_data['neighborhood'] = np.nan
for i in range(0,house_data.shape[0]):
    house_data.loc[i,'neighborhood']= house_data.loc[i,'data_url'].split('/')[5]

# remove  text in columns ['beds','baths','sqft',''price_$CAN]
house_data.beds = house_data.beds.apply(lambda x: re.sub('\D','',x))
house_data.baths = house_data.baths.apply(lambda x: re.sub('\D','',x))
house_data['price_$CAN'] = house_data['price_$CAN'].apply(lambda x: re.sub('\D','',x) )
house_data.sqft = house_data.sqft.apply(lambda x: re.sub('\D','',x))

#rename the price_$CAN to price_dallar_can
house_data.rename(columns ={'price_$CAN':'price_100K'},inplace =True)

# remove (-) in neignborhood columns
house_data.neighborhood = house_data.neighborhood.str.replace('-',' ')


# drop columns ['data_url','Unnamed: 0']
house_data.drop(columns =['data_url','Unnamed: 0'],axis=1,inplace =True)


(3525, 10)


In [32]:
# filter null value of ['beds','baths','sqft']
house_data = house_data.loc[house_data['beds']!= '' ]
house_data = house_data.loc[house_data['baths']!= '' ]
house_data = house_data.loc[house_data['sqft']!= '' ]
house_data[['price_100K','beds','baths','sqft']] = house_data[['price_100K','beds','baths','sqft']].astype('int')
house_data['price_100K'] = house_data['price_100K']/100000.0
print(house_data.shape)
house_data.head(10)

(947, 9)


,address,long,lat,price_100K,beds,baths,sqft,type,neighborhood
0,"33 Shore Breeze Dr, Toronto, Ontario",-79.478979,43.623426,6.4900,1,1,699,Condo,Humber Bay
1,"1151 Markham Rd., Toronto, Ontario",-79.231107,43.776168,5.4299,1,1,678,Condo,Curran Hall
4,"7 Bishop Ave, Toronto, Ontario",-79.414311,43.781737,6.3700,2,1,1399,Condo,Newtonbrook
6,"Lake & Town Urban Townhomes, Etobicoke, Ontario",-79.565600,43.643600,6.2500,2,2,900,Townhouse,Bloordale West Mall
7,"36 Park Lawn Rd, Toronto, Ontario",-79.484151,43.623604,5.3900,1,1,599,Condo,Mimico
9,"Sky Tower One Yonge Condos - 11 Yonge Street, ...",-79.384900,43.652900,7.6990,1,1,550,Apartment,Bay Street Corridor
13,"20 Tubman Ave, Toronto, Ontario",-79.359362,43.660605,5.9500,1,1,730,Condo,Regent Park Trefann Court
15,"955 Bay Street 8TH, Toronto, Ontario",-79.386980,43.664777,6.7000,1,1,583,Condo,Bay Street Corridor
17,"30 Ordnance St, Toronto, Ontario",-79.409769,43.639732,6.9900,1,1,699,Condo,Fort York
22,"250 Manitoba St, Toronto, Ontario",-79.489962,43.624447,5.6790,1,1,799,Condo,Mimico


In [61]:
type_df =pd.get_dummies(house_data.type)
type_df.head()
house_data['Apartment']=type_df.Apartment
house_data['Condo']=type_df.Condo
house_data['Residential']=type_df.Residential
house_data['Townhouse']=type_df.Townhouse
neighborhood_house_data = np.round(house_data.groupby(['neighborhood']).mean(),2)
neighborhood_house_data.reset_index(inplace =True)
neighborhood_house_data

,neighborhood,long,lat,price_100K,beds,baths,sqft,Apartment,Condo,Residential,Townhouse
0,Bay Street Corridor,-79.39,43.65,7.81,1.05,1.05,572.70,0.94,0.02,0.05,0.0
1,Bayview Village,-79.38,43.77,5.47,1.33,1.50,702.00,0.50,0.33,0.17,0.0
2,Bloordale West Mall,-79.57,43.64,6.25,2.00,2.00,900.00,0.00,0.00,0.00,1.0
3,Casa Loma,-79.42,43.68,6.29,1.00,1.00,699.00,0.00,0.00,1.00,0.0
4,Christie Pits,-79.42,43.67,17.99,3.00,5.00,2500.00,0.00,0.00,1.00,0.0
5,Cityplace,-79.39,43.64,6.33,1.11,1.11,665.67,0.00,0.22,0.78,0.0
6,Curran Hall,-79.23,43.78,5.43,1.00,1.00,678.00,0.00,1.00,0.00,0.0
7,Distillery District,-79.36,43.65,10.95,2.00,2.00,1199.00,0.00,0.00,1.00,0.0
8,Don Mills,-79.34,43.75,16.89,4.00,2.00,2090.00,0.00,0.00,1.00,0.0
9,Downsview,-79.47,43.73,13.48,3.50,2.50,1849.50,0.00,0.00,1.00,0.0


In [62]:
mci_2014_2018 = pd.read_csv('./dataset/mci_2014_to_2018.csv')
print(mci_2014_2018.shape)
mci_2014_2018['Neighbourhood'] = mci_2014_2018.Neighbourhood.apply(lambda x: x.split('(')[0])
mci_2014_2018.head()

(167525, 29)


,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,reportedyear,reportedmonth,reportedday,reporteddayofyear,reporteddayofweek,reportedhour,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Lat,Long,ObjectId
0,-79.385193,43.659229,2349,GO-20149004286,2014-06-20T10:55:00.000Z,2014-06-20T13:20:00.000Z,Apartment,2130,210,Theft Over,2014,June,20,171,Friday,13,2014.0,June,20.0,171.0,Friday,10,Theft Over,D52,76,Bay Street Corridor,43.659229,-79.385193,2001
1,-79.425400,43.777592,2350,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1457,100,Pointing A Firearm,2014,July,2,183,Wednesday,2,2014.0,July,2.0,183.0,Wednesday,0,Assault,D32,36,Newtonbrook West,43.777592,-79.425400,2002
2,-79.425400,43.777592,2351,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1610,100,Robbery With Weapon,2014,July,2,183,Wednesday,2,2014.0,July,2.0,183.0,Wednesday,0,Robbery,D32,36,Newtonbrook West,43.777592,-79.425400,2003
3,-79.210373,43.801727,2352,GO-20142412127,2014-07-02T01:30:00.000Z,2014-07-02T05:40:00.000Z,House,2120,200,B&E,2014,July,2,183,Wednesday,5,2014.0,July,2.0,183.0,Wednesday,1,Break and Enter,D42,132,Malvern,43.801727,-79.210373,2004
4,-79.254334,43.835884,2354,GO-20142417548,2014-07-02T20:52:00.000Z,2014-07-02T20:57:00.000Z,Commercial,1430,100,Assault,2014,July,2,183,Wednesday,20,2014.0,July,2.0,183.0,Wednesday,20,Assault,D42,130,Milliken,43.835884,-79.254334,2005


In [63]:
mci_2014_2018.head(10)
MCI = pd.get_dummies(mci_2014_2018.MCI)
MCI.shape

(167525, 5)

In [64]:
day = pd.get_dummies(mci_2014_2018.occurrencedayofweek)
day.head()

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1


In [65]:
hours = pd.get_dummies(mci_2014_2018.occurrencehour)
day.head()

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1


In [69]:
columns=['Long','Lat','Neighbourhood']
for i in day.columns.values:
    mci_2014_2018[i] = day[i]
    columns.append(i)
    

for i in hours.columns.values:
    mci_2014_2018[i] = hours[i]
    columns.append(i)

for i in MCI.columns.values:
    mci_2014_2018[i] = MCI[i]
    columns.append(i)
    
mci_2014_2018 = mci_2014_2018[columns]
mci_2014_2018.groupby(['Neighbourhood']).mean()

,Long,Lat,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,Assault,Auto Theft,Break and Enter,Robbery,Theft Over
Neighbourhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt North,-79.273078,43.806642,0.181548,0.152778,0.116071,0.089286,0.163690,0.137897,0.158730,0.064484,0.033730,0.021825,0.024802,0.014881,0.015873,0.005952,0.016865,0.036706,0.046627,0.041667,0.053571,0.065476,0.055556,0.050595,0.055556,0.050595,0.040675,0.055556,0.069444,0.060516,0.041667,0.050595,0.026786,0.388889,0.131944,0.291667,0.160714,0.026786
Agincourt South-Malvern West,-79.269693,43.787151,0.166792,0.132231,0.160030,0.120210,0.132983,0.145755,0.141247,0.081142,0.035312,0.054846,0.034560,0.027047,0.027047,0.009767,0.018032,0.038317,0.035312,0.051089,0.038317,0.057851,0.033809,0.040571,0.041322,0.050338,0.047333,0.057851,0.042825,0.042074,0.045079,0.047333,0.042825,0.435763,0.119459,0.286251,0.111195,0.047333
Alderwood,-79.541881,43.605487,0.165217,0.154348,0.154348,0.126087,0.119565,0.141304,0.139130,0.063043,0.043478,0.067391,0.034783,0.030435,0.019565,0.015217,0.021739,0.030435,0.028261,0.021739,0.021739,0.047826,0.041304,0.050000,0.030435,0.045652,0.032609,0.069565,0.047826,0.052174,0.050000,0.078261,0.056522,0.395652,0.182609,0.269565,0.078261,0.073913
Annex,-79.403475,43.669477,0.162311,0.114512,0.177442,0.157153,0.134801,0.119326,0.134457,0.064649,0.061554,0.080468,0.051926,0.033012,0.023040,0.016506,0.012724,0.022352,0.026823,0.030261,0.033356,0.054333,0.040578,0.037139,0.051238,0.044360,0.043329,0.046768,0.045736,0.048831,0.043673,0.046424,0.040922,0.523728,0.039546,0.268913,0.107978,0.059835
Banbury-Don Mills,-79.344994,43.734562,0.169510,0.138593,0.160981,0.106610,0.116205,0.148188,0.159915,0.057569,0.026652,0.033049,0.019190,0.019190,0.008529,0.020256,0.025586,0.033049,0.040512,0.040512,0.036247,0.063966,0.040512,0.068230,0.052239,0.047974,0.055437,0.061834,0.072495,0.049041,0.050107,0.041578,0.036247,0.424307,0.089552,0.363539,0.074627,0.047974
Bathurst Manor,-79.456367,43.762388,0.148265,0.177708,0.117771,0.141956,0.123028,0.141956,0.149317,0.069401,0.038906,0.030494,0.023134,0.016824,0.011567,0.014721,0.018927,0.034700,0.039958,0.046267,0.052576,0.067298,0.031546,0.047319,0.050473,0.051525,0.045216,0.056782,0.051525,0.046267,0.055731,0.060988,0.037855,0.438486,0.182965,0.252366,0.072555,0.053628
Bay Street Corridor,-79.384198,43.654395,0.159652,0.136083,0.148990,0.145342,0.141414,0.127946,0.140572,0.056958,0.046857,0.046577,0.032828,0.022166,0.023569,0.015152,0.023008,0.030303,0.043771,0.048541,0.045735,0.054153,0.046016,0.051066,0.052469,0.055836,0.045455,0.049383,0.044893,0.036476,0.037879,0.045455,0.045455,0.686308,0.030584,0.150954,0.080247,0.051908
Bayview Village,-79.376377,43.772382,0.164160,0.117794,0.162907,0.129073,0.149123,0.146617,0.130326,0.062657,0.026316,0.023810,0.023810,0.008772,0.011278,0.010025,0.032581,0.043860,0.037594,0.045113,0.035088,0.070175,0.055138,0.055138,0.043860,0.068922,0.045113,0.061404,0.053885,0.048872,0.050125,0.048872,0.037594,0.520050,0.110276,0.271930,0.047619,0.050125
Bayview Woods-Steeles,-79.382970,43.795946,0.135514,0.144860,0.121495,0.119159,0.175234,0.151869,0.151869,0.056075,0.028037,0.014019,0.025701,0.021028,0.025701,0.021028,0.023364,0.051402,0.051402,0.053738,0.042056,0.100467,0.053738,0.044393,0.049065,0.030374,0.030374,0.058411,0.053738,0.051402,0.030374,0.056075,0.028037,0.420561,0.142523,0.366822,0.044393,0.025701


## 2.3 Feature selection

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,Assault,Auto Theft,Break and Enter,Division,Hood_ID,Index_,Lat,Long,MCI,Neighbourhood,ObjectId,Robbery,Theft Over,X,Y,event_unique_id,occurrencedate,occurrenceday,occurrencedayofweek,occurrencedayofyear,occurrencehour,occurrencemonth,occurrenceyear,offence,premisetype,reporteddate,reportedday,reporteddayofweek,reporteddayofyear,reportedhour,reportedmonth,reportedyear,ucr_code,ucr_ext
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
mci_2014_2018 = pd.read_csv('./dataset/mci_2014_to_2018.csv')
print(mci_2014_2018.shape)
mci_2014_2018.head()

(167525, 29)


,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,reportedyear,reportedmonth,reportedday,reporteddayofyear,reporteddayofweek,reportedhour,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Lat,Long,ObjectId
0,-79.385193,43.659229,2349,GO-20149004286,2014-06-20T10:55:00.000Z,2014-06-20T13:20:00.000Z,Apartment,2130,210,Theft Over,2014,June,20,171,Friday,13,2014.0,June,20.0,171.0,Friday,10,Theft Over,D52,76,Bay Street Corridor (76),43.659229,-79.385193,2001
1,-79.425400,43.777592,2350,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1457,100,Pointing A Firearm,2014,July,2,183,Wednesday,2,2014.0,July,2.0,183.0,Wednesday,0,Assault,D32,36,Newtonbrook West (36),43.777592,-79.425400,2002
2,-79.425400,43.777592,2351,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1610,100,Robbery With Weapon,2014,July,2,183,Wednesday,2,2014.0,July,2.0,183.0,Wednesday,0,Robbery,D32,36,Newtonbrook West (36),43.777592,-79.425400,2003
3,-79.210373,43.801727,2352,GO-20142412127,2014-07-02T01:30:00.000Z,2014-07-02T05:40:00.000Z,House,2120,200,B&E,2014,July,2,183,Wednesday,5,2014.0,July,2.0,183.0,Wednesday,1,Break and Enter,D42,132,Malvern (132),43.801727,-79.210373,2004
4,-79.254334,43.835884,2354,GO-20142417548,2014-07-02T20:52:00.000Z,2014-07-02T20:57:00.000Z,Commercial,1430,100,Assault,2014,July,2,183,Wednesday,20,2014.0,July,2.0,183.0,Wednesday,20,Assault,D42,130,Milliken (130),43.835884,-79.254334,2005


# 3. Exploratory Data Analysis

# 3.1 Calculation of target variable